# 7-Slot Nerdle Solver Test
We prove (by brute-force) that you can always solve mini-Nerdle in at most $4$ guesses regardless of the starting expression, provided you use the optimal strategy. The worst start having repeating numbers and thus less information, e.g. `10-5=5`. The best start has all different numbers: `28/7=4`, which needs at most $3$ guesses and $2.65 \pm 0.5$ guesses.

In [2]:
%load_ext autoreload
%autoreload 2

import collections
import itertools
import numpy as np
import matplotlib.pyplot as plt

import nerdle
import score as s
import generator
from nerdle import Hint, NerdleData

In [4]:
# Mini-Nerdle.
NUM_SLOTS = 7
SCORE_DICT_FILE = "nerdle{}.db".format(NUM_SLOTS) 

solver_data = nerdle.create_solver_data(NUM_SLOTS, SCORE_DICT_FILE)

In [5]:
d = solver_data.score_dict
print(len(d), len(solver_data.answers))
for key in list(d.keys())[:10]:
    print(key, "".join(map(str, key[0])) + "=" + str(key[1]), len(d[key]))

6661 6661
98-4=94 9=8 6661
7-8+7=6 7=- 6661
75-3=72 7=5 6661
8-2*1=6 8=- 6661
51+5=56 5=1 6661
83-2=81 8=3 6661
23-20=3 2=3 6661
68/1=68 6=8 6661
9+5-9=5 9=+ 6661
94/47=2 9=4 6661


## Example Usage

In [11]:
score = nerdle.score_guess("1*1*6=6", "99/9=11")
print(score, 
      s.score_to_hint_string(score, NUM_SLOTS),
      s.score_to_hints(score, NUM_SLOTS) == [Hint.MISPLACED, Hint.INCORRECT, Hint.MISPLACED, Hint.INCORRECT, Hint.INCORRECT, Hint.MISPLACED, Hint.INCORRECT]
     )

2082 ?-?--?- True


In [20]:
# A good initial guess significantly reduces the number of answers. In this case, from
# 206 to 10.
guess_history, hint_history, answer_size_history =  nerdle.NerdleSolver(solver_data).solve("99/9=11", initial_guess= "1+6-7=0", debug=True)

guess 1+6-7=0 score ?----?- answers 128
guess 2*19=38 score --?++-- answers 1
guess 99/9=11 score +++++++ answers 1


## Benchmark
This is a fast in-memory dict implementation.

In [7]:
%time guess_history, hint_history, answer_size_history = nerdle.NerdleSolver(solver_data).solve("99/9=11", initial_guess="1+6-7=0")

CPU times: user 2.05 s, sys: 10.9 ms, total: 2.06 s
Wall time: 2.06 s


In [ ]:
answer = "99/9=11"
solver = nerdle.NerdleSolver(solver_data)
for start in list(solver_data.answers)[:5]:
    print(start)
    %time solver.solve(answer, initial_guess=start) 

## Initial Guess Optimization
Assuming something with a lot of different numbers and operators is best.

In [ ]:
# answer = "99/9=11"
# solutions = [nerdle.NerdleSolver(solver_data).solve(answer, initial_guess=start) 
#              for start in solver_data.answers]
# n = np.array([len(solution[0]) for solution in solutions])
# num_answers = len(solver_data.answers)
# compression_ratio = num_answers / np.array([solution[2][0] for solution in solutions])
# print(np.mean(compression_ratio), np.std(compression_ratio))

In [17]:
start = "1+6-7=0"
solutions = [nerdle.NerdleSolver(solver_data).solve(answer, initial_guess=start) 
             for answer in solver_data.answers]               
n = np.array([len(solution[0]) for solution in solutions])
num_answers = len(solver_data.answers)
compression_ratio = num_answers / np.array([solution[2][0] for solution in solutions])
print(np.mean(compression_ratio), np.std(compression_ratio))

fig, axs = plt.subplots(1, 2, figsize=(10, 4))

ax = axs[0]
ax.hist(n);
ax.set_title("#Guesses for start {}".format(start));

ax = axs[1]
ax.hist(compression_ratio);
ax.set_title("Compression Ratio Distribution, start {}".format(start));

KeyboardInterrupt: 

## Profiling

In [11]:
import cProfile
import pstats
from pstats import SortKey

cProfile.run('guess_history, hint_history, answer_size_history = nerdle.NerdleSolver(solver_data).solve("99/9=11", initial_guess="1+6-7=0", debug=True)', 'stats')
p = pstats.Stats('stats')
p.sort_stats(SortKey.CUMULATIVE).print_stats(20)

--> guess 1+6-7=0
score ?----?- 2050 answers 128
--> guess 2*19=38
score --?++-- 352 answers 1
--> guess 99/9=11
score +++++++ 5461 answers 1
Wed Sep 14 07:53:09 2022    stats

         1026054 function calls (1026033 primitive calls) in 4.916 seconds

   Ordered by: cumulative time
   List reduced from 60 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.916    4.916 {built-in method builtins.exec}
        1    0.001    0.001    4.916    4.916 <string>:1(<module>)
        1    0.000    0.000    4.913    4.913 /Users/olivne/oren/nerdle-solver/nerdle.py:158(solve)
        1    0.001    0.001    4.913    4.913 /Users/olivne/oren/nerdle-solver/nerdle.py:163(solve_adversary)
        3    0.020    0.007    4.907    1.636 /Users/olivne/oren/nerdle-solver/nerdle.py:192(make_guess)
        1    0.000    0.000    4.548    4.548 /Users/olivne/oren/nerdle-solver/nerdle.py:75(restrict_by_answers)
        1    0.2

In [17]:
n = len(solver_data.answers)

In [19]:
%time a = np.zeros((n, n))

CPU times: user 151 µs, sys: 669 µs, total: 820 µs
Wall time: 656 µs


In [20]:
a.shape

(6661, 6661)